In [ ]:
batch_size = 8
source_folder = '/content/drive/MyDrive/Dataset/Morteza/Data/Resnet/DataMain/MortezaRawImages'
metal_indicator_lung = '/content/drive/MyDrive/Dataset/Morteza/Data/Resnet/Test/Metals_Indicators.xlsx'
keyFileNameCorr = '/content/drive/MyDrive/Dataset/Morteza/Data/Resnet/Test/Lung1-7.xlsx'
clinical_var_address = '/content/drive/MyDrive/Dataset/Morteza/Data/Resnet/Test/Variables/progressNotProgres.xlsx'
dir_experiment = '/content/drive/MyDrive/Dataset/Morteza/Data/Resnet/Experiments/'
experiment_name = 'pred_progress_v_0'
channel_names = ['aSMA','Arg1','B7-H3','B7-H4','BCL2','CD3','CD4','CD8a','CD11c','CD14','CD16','CD20','CD31','CD39','CD40','CD45','CD68','CD94','CD117','CD163','Cleaved Caspase3','DNA1','DNA2','FoxP3','HIF-1a','Histone H3','HLA-DR','Ki67','MCSFR','MMP9','MPO','Pancytokeratin','PD1','PDL1-Biotin','pERK','pSTAT3','TTF1']
RAW_Image_Source_Folder = '/content/drive/MyDrive/Dataset/Morteza/Data/Resnet/DataMain/MortezaRawImages/'
num_classes = 2


In [ ]:
import torchvision
import torch.nn as nn
from functools import partial
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

resnet_models = {18: torchvision.models.resnet18,
                 34: torchvision.models.resnet34,
                 50: torchvision.models.resnet50,
                 101: torchvision.models.resnet101,
                 152: torchvision.models.resnet152}

class Resnet_multichannel(nn.Module):
    def __init__(self, pretrained=True, encoder_depth=34, num_in_channels=4):
        super().__init__()
        
        if encoder_depth not in [18, 34, 50, 101, 152]:
            raise ValueError(f"Encoder depth {encoder_depth} specified does not match any existing Resnet models")
            
        model = resnet_models[encoder_depth](pretrained)
        
        ##For reference: layers to use (in order):
        # conv1, bn1, relu, maxpool, layer1, layer2, layer3, layer4, avgpool, fc
        
        # This is the most important line of code here. This increases the number of in channels for our network
        self.conv1 = self.increase_channels(model.conv1, num_in_channels)
        
        self.bn1 = model.bn1
        self.relu = model.relu
        self.maxpool = model.maxpool
        self.layer1 = model.layer1
        self.layer2 = model.layer2
        self.layer3 = model.layer3
        self.layer4 = model.layer4
        self.avgpool = model.avgpool
        self.fc = model.fc
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
        
    def increase_channels(self, m, num_channels=None, copy_weights=0):


        """
        takes as input a Conv2d layer and returns the a Conv2d layer with `num_channels` input channels
        and all the previous weights copied into the new layer.
        """
        # number of input channels the new module should have
        new_in_channels = num_channels if num_channels is not None else m.in_channels + 1
        
        bias = False if m.bias is None else True
        
        # Creating new Conv2d layer
        new_m = nn.Conv2d(in_channels=new_in_channels, 
                          out_channels=m.out_channels, 
                          kernel_size=m.kernel_size, 
                          stride=m.stride, 
                          padding=m.padding,
                          bias=bias)
        
        # Copying the weights from the old to the new layer
        new_m.weight.data[:, :m.in_channels, :, :] = m.weight.clone()
        
        #Copying the weights of the `copy_weights` channel of the old layer to the extra channels of the new layer
        for i in range(new_in_channels - m.in_channels):
            channel = m.in_channels + i
            new_m.weight.data[:, channel:channel+1, :, :] = m.weight[:, copy_weights:copy_weights+1, : :].clone()
        new_m.weight = nn.Parameter(new_m.weight)

        return new_m
    
def get_arch(encoder_depth, num_in_channels):
    """
    Returns just an architecture which can then be called in the usual way.
    For example:
    resnet34_4_channel = get_arch(34, 4)
    model = resnet34_4_channel(True)
    """
    return partial(Resnet_multichannel, encoder_depth=encoder_depth, num_in_channels=num_in_channels)

In [ ]:
resnet50_mchannel = get_arch(50, 3000)
model = resnet50_mchannel(True) 
print("New input channels : ", model.conv1.in_channels)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

New input channels :  3000


In [ ]:
!pip install torchinfo
from torchinfo import summary


In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
feature_extract = True
set_parameter_requires_grad(model, feature_extract)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
input_size = 224

In [ ]:
print(summary(model))

Layer (type:depth-idx)                   Param #
Resnet_multichannel                      --
├─Conv2d: 1-1                            (9,408,000)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  (4,096)
│    │    └─BatchNorm2d: 3-2             (128)
│    │    └─Conv2d: 3-3                  (36,864)
│    │    └─BatchNorm2d: 3-4             (128)
│    │    └─Conv2d: 3-5                  (16,384)
│    │    └─BatchNorm2d: 3-6             (512)
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              (16,896)
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  (16,384)
│    │    └─BatchNorm2d: 3-10            (128)
│    │    └─Conv2d: 3-11                 (36,864)
│    │    └─BatchNorm2d: 3-12            (128)
│    │   

In [ ]:
# test for inference
import PIL.Image as Image
import torchvision.transforms as transforms
to_tensor = transforms.ToTensor()

do_test = 0
if do_test:
  for i in range(19):
    if i==0:
      im = Image.open('/content/drive/MyDrive/Dataset/Morteza/Data/Sample_Data/LungTest0/1/1.png')
      im = im.resize([224,224])
      image = to_tensor(im)
    else:
      # print('/content/data_sample/'+str(i+1)+'.png')
      im = Image.open('/content/drive/MyDrive/Dataset/Morteza/Data/Sample_Data/LungTest0/1/'+str(i+1)+'.png')
      im = im.resize([224,224])
      next_channel = to_tensor(im)
      image = torch.cat((image,next_channel),0);  
  print(image.shape)
  # image = torch.unsqueeze(image, 1)
  uimage = torch.unsqueeze(image, 0)
  model.eval()
  out = model(uimage)
  print(out)

In [ ]:
# Send the model to GPU
# Detect if we have a GPU available
import torch.optim as optim
model = model.to(device)
params_to_update = model.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [ ]:
import time
import copy

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == 'train':                      
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from pathlib import Path
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from glob import glob
import numpy as np


In [ ]:
allExamples = glob(source_folder+"/*/", recursive = True)


In [ ]:
!pip install --upgrade openpyxl
df = pd.read_excel(metal_indicator_lung)
channel_names = df.Indicator.values
co_marker = df.Comarker.values
num_channels = len(channel_names)
print(num_channels)
allResponses = np.zeros((len(allExamples),num_channels*2048))

37


In [ ]:
def read_variable_file_check_examples(file_path,mother_examples):
  df = pd.read_excel(file_path, dtype=str)
  df_key_filename = pd.read_excel(keyFileNameCorr, dtype=str)
  num_classes = len(df.columns.values)
  counter = 0
  aFNs = []
  aFNLabels = []
  for i in range(num_classes):
    
    cur_column = df[df.columns[i]].values
    for key in cur_column:    
      if type(key) is str and key!='nan':
        
        ind_num = [index for index, value in enumerate(df_key_filename["UniqueImageID"].values) if value == key]
        if len(ind_num)!=0:
          counter = counter + 1        
          ind_num = ind_num[0]
          # print(key,', ',df_key_filename["FileName"][ind_num])

          existInExampleSet = -1
          counter_eiE = 0
          query_str = df_key_filename["FileName"][ind_num]
          while(existInExampleSet!=1) and counter_eiE < len(mother_examples):
            if query_str in mother_examples[counter_eiE]:
              existInExampleSet = 1
            counter_eiE = counter_eiE + 1

          if existInExampleSet==1:
            aFNs.append(df_key_filename["FileName"][ind_num])
            aFNLabels.append(i)

  return aFNs,aFNLabels


aFileNames,aFileNamesLabels = read_variable_file_check_examples(clinical_var_address,allExamples)


In [ ]:
from sklearn.model_selection import train_test_split
aFileNames_train, aFileNames_test, aFileNamesLabels_train, aFileNamesLabels_test = train_test_split(aFileNames, aFileNamesLabels, test_size=0.2, random_state=42)

In [ ]:
import os


dir1 = dir_experiment+experiment_name
dir2 = dir_experiment+experiment_name+"/train/"
dir3 = dir_experiment+experiment_name+"/val/"

used = set()
uniqueLabels = [x for x in aFileNamesLabels if x not in used and (used.add(x) or True)]


if not os.path.exists(dir_experiment):
  os.mkdir(dir_experiment)



if not os.path.exists(dir1):
  os.mkdir(dir1)
if not os.path.exists(dir2):
  os.mkdir(dir2)
if not os.path.exists(dir3):
  os.mkdir(dir3)


for j in range(len(uniqueLabels)):
  c_dir = dir2+str(uniqueLabels[j])+'/'
  if not os.path.exists(c_dir):
    os.mkdir(c_dir)
for j in range(len(uniqueLabels)):
  c_dir = dir3+str(uniqueLabels[j])+'/'
  if not os.path.exists(c_dir):
    os.mkdir(c_dir)

for i in range(len(aFileNames_train)):
  file = dir2+str(aFileNamesLabels_train[i])+'/'+aFileNames_train[i]+'.tiff'
  open(file, 'a').close()

for i in range(len(aFileNames_test)):
  file = dir3+str(aFileNamesLabels_test[i])+'/'+aFileNames_test[i]+'.tiff'
  open(file, 'a').close()

In [ ]:


def my_tiff_loader(filename):
  head_tail = os.path.split(filename)
  filenameWithTiff = head_tail[-1]
  
  fileNameToBeUsed = filenameWithTiff[0:-(len('.tiff'))]
  folder_path = RAW_Image_Source_Folder+fileNameToBeUsed+'/'
  for i in range(len(channel_names)):
    if i==0:
      im = Image.open(folder_path+channel_names[i]+'.png')
      image = im.resize([224,224])
      image = np.expand_dims(image,0).astype(np.float32)
    else:
      im = Image.open(folder_path+channel_names[i]+'.png')
      next_channel = im.resize([224,224])
      next_channel = np.expand_dims(next_channel,0).astype(np.float32)
      image = np.concatenate((image,next_channel),0)

  return image

from torchvision import datasets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
    ]),
}
image_datasets = {x: datasets.ImageFolder(os.path.join(dir1, x),loader=my_tiff_loader) for x in ['train', 'val']}
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
